In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# define model
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [7]:
# apply normal distribution to model's weight
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)                   # Cross Entropy Loss
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)  # Adam optimizer

In [9]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)  # batch_size, 784
        Y = Y.to(device)                    # batch_size
        
        hypothesis = linear(X)              # batch_size, 10
        cost = criterion(hypothesis, Y)     # single value
        
        # backward
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '%.9f' % avg_cost)
    
print('Learning finished')

Epoch: 0001 cost = 5.656054974
Epoch: 0002 cost = 1.699803472
Epoch: 0003 cost = 1.121566176
Epoch: 0004 cost = 0.883357942
Epoch: 0005 cost = 0.750486851
Epoch: 0006 cost = 0.663246810
Epoch: 0007 cost = 0.601343870
Epoch: 0008 cost = 0.554267764
Epoch: 0009 cost = 0.518746793
Epoch: 0010 cost = 0.489619613
Epoch: 0011 cost = 0.465846598
Epoch: 0012 cost = 0.446371138
Epoch: 0013 cost = 0.429083288
Epoch: 0014 cost = 0.414352983
Epoch: 0015 cost = 0.401432723
Learning finished


In [10]:
"""
torchvision.datasets.test_data has been renamed data
torchvision.datasets.test_labels has been renamed targets
"""
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)  # batch_size, 784
    Y_test = mnist_test.targets.to(device)                         # batch_size
    
    prediction = linear(X_test)                                 # batch_size, 10
    correct_prediction = torch.argmax(prediction, 1) == Y_test  # batch_size
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_pred = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_pred, 1).item())

Accuracy:  0.8905999660491943
Label:  8
Prediction:  3
